Load the data

In [ ]:
from pandas import json_normalize
import json
# load JSON-file
with open('dataset.json', 'r') as file:
    data = json.load(file)
    # normalize data (return data frame)
    df = json_normalize(data)